In [8]:
import os
from tifffile import imread, imwrite
import numpy as np

file_path = r'D:\Projects\Denoising-STED\20220913-RPI\photobleaching\files'
save_path = r'D:\Projects\Denoising-STED\20220913-RPI\photobleaching\patches'

list_files = os.listdir(file_path)
M = len(list_files)
patch_size = 800
threshold = 0.0
count1 = 0
for i in list_files:
    f = imread(os.path.join(file_path, '', i)).astype(np.float32)
    f = f / f.max()
    f = f - f.min()
    f = f / f.max()
    f[f < 0] = 0
    w = f.shape[-1]
    h = f.shape[-2]
    if len(f.shape) == 3:
        t = f.shape[0]
    else:
        t = 1
        f = f.reshape((1, f.shape[0], f.shape[1]))
        
#     nx = int(3*np.ceil(h / patch_size))
#     ny = int(3*np.ceil(w / patch_size))
    nx = 3
    ny = 3
    
    x = np.zeros((nx * ny * t, patch_size, patch_size))
    nnx = np.floor(np.linspace(0, h - patch_size, nx)).astype(np.int32)
    nny = np.floor(np.linspace(0, w - patch_size, ny)).astype(np.int32)
#     print(nnx,nny)
    count = 0
    for p in range(nx):
        for q in range(ny):
            for j in range(t):
                x[count, :, :] = f[j, nnx[p]:nnx[p]+patch_size, nny[q]:nny[q]+patch_size]
                x[count] = x[count]/x[count].max()
                count = count + 1

#     norm_x = np.linalg.norm(x, axis=(1, 2))
#     norm_x = norm_x / norm_x.max()
#     ind_norm = np.where(norm_x > threshold)[0]
#     y = np.zeros((len(ind_norm), patch_size, patch_size))
#     for k in range(len(ind_norm)):
#         y[k] = x[ind_norm[k]]
    y = np.uint16(x * (2 ** 16 - 1))
    imwrite(os.path.join(save_path, '', str(count1)+'.tif'), y, imagej=True, metadata={'axes': 'TYX'})
    count1 = count1+1


In [1]:
import numpy as np

In [16]:
x = np.random.rand(2,3,3)

In [17]:
mx = x.max(axis=(1,2))

In [18]:
mx = mx.reshape((2,1,1))

In [19]:
nx = x/mx

In [25]:
nx[0,:,:]

array([[0.47591327, 0.09682424, 0.91850126],
       [0.58563143, 0.98365645, 0.53816074],
       [0.80308317, 0.90736947, 1.        ]])

In [27]:
x[0,:,:]/0.82241154

array([[0.47591327, 0.09682424, 0.91850127],
       [0.58563144, 0.98365645, 0.53816074],
       [0.80308317, 0.90736947, 1.        ]])